In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import re

In [8]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
 
#s = Service("/Users/jiacheng/Documents/MacBookPro/Academic/iMep/sns/SNS\ download/chromedriver")
#driver = webdriver.Chrome(service=s)
driver = webdriver.Chrome('/Users/jiacheng/Downloads/chromedriver')
#driver.get('https://www.baidu.com')
#driver.close()



In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time
import re

# 根据时间段、翻页爬取
def Due_Time(year, month):
    #s = Service("/Users/jiacheng/chromedriver")
    #browser = webdriver.Chrome(service=s)
    browser = webdriver.Chrome()
    url = 'http://sns.sseinfo.com/qa.do'

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
         AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}

    browser.get(url)
    # browser.maximize_window()  # 将模拟浏览器窗口最大化，以显示登录框

    # 去掉这个才能写入日期（小细节）
    js = 'document.getElementById("sdate").removeAttribute("readonly")'
    browser.execute_script(js)
    js = 'document.getElementById("edate").removeAttribute("readonly")'
    browser.execute_script(js)

    # 按时间段检索
    if month == 9:
        #browser.find_element_by_xpath('//*[@id="sdate"]').send_keys(f'{year}-09-01')
        browser.find_element(By.XPATH, '//*[@id="sdate"]').send_keys(f'{year}-09-01')
        #browser.find_element_by_xpath('//*[@id="edate"]').send_keys(f'{year}-10-01')
        browser.find_element(By.XPATH, '//*[@id="edate"]').send_keys(f'{year}-10-01')
        #browser.find_element_by_xpath('//*[@id="container"]/div[1]/div[2]/div[1]/div/div[2]/div[2]').click()
        browser.find_element(By.XPATH, '//*[@id="container"]/div[1]/div[2]/div[1]/div/div[2]/div[2]').click()
        time.sleep(7)
    if month == 10 or month == 11:
        #browser.find_element_by_xpath('//*[@id="sdate"]').send_keys(f'{year}-{month}-01')
        browser.find_element(By.XPATH, '//*[@id="sdate"]').send_keys(f'{year}-{month}-01')
        #browser.find_element_by_xpath('//*[@id="edate"]').send_keys(f'{year}-{month+1}-01')
        browser.find_element(By.XPATH, '//*[@id="edate"]').send_keys(f'{year}-{month+1}-01')
        #browser.find_element_by_xpath('//*[@id="container"]/div[1]/div[2]/div[1]/div/div[2]/div[2]').click()
        browser.find_element(By.XPATH, '//*[@id="container"]/div[1]/div[2]/div[1]/div/div[2]/div[2]').click()
        time.sleep(7)
    if month == 12:
        #browser.find_element_by_xpath('//*[@id="sdate"]').send_keys(f'{year}-{month}-01')
        browser.find_element(By.XPATH, '//*[@id="sdate"]').send_keys(f'{year}-{month}-01')
        #browser.find_element_by_xpath('//*[@id="edate"]').send_keys(f'{year}-{month}-31')
        browser.find_element(By.XPATH, '//*[@id="edate"]').send_keys(f'{year}-{month}-31')
        #browser.find_element_by_xpath('//*[@id="container"]/div[1]/div[2]/div[1]/div/div[2]/div[2]').click()
        browser.find_element(By.XPATH, '//*[@id="container"]/div[1]/div[2]/div[1]/div/div[2]/div[2]').click()
        time.sleep(7)
    if month < 9 :
        #browser.find_element_by_xpath('//*[@id="sdate"]').send_keys(f'{year}-0{month}-01')
        browser.find_element(By.XPATH, '//*[@id="sdate"]').send_keys(f'{year}-0{month}-01')
        #browser.find_element_by_xpath('//*[@id="edate"]').send_keys(f'{year}-0{month+1}-01')
        browser.find_element(By.XPATH, '//*[@id="edate"]').send_keys(f'{year}-0{month+1}-01')
        #browser.find_element_by_xpath('//*[@id="container"]/div[1]/div[2]/div[1]/div/div[2]/div[2]').click()
        browser.find_element(By.XPATH, '//*[@id="container"]/div[1]/div[2]/div[1]/div/div[2]/div[2]').click()
        time.sleep(7)

    to_csv = open(f'C:\\Users\\xc172\\Downloads\\SNS download\\上证e互动{year}_{month}.csv', 'w', encoding='utf-8', newline="")
    # 提问时间抓不到
    l = ['股票名称', '股票代码', '提问人名', '提问人ID', '提问人所在地', '提问文本', '答复时间', '答复文本']
    tmpstr = ','.join(l) + '\n'
    to_csv.write(tmpstr)

    page = 0
    while 1:
        page += 1
        print(page)

        # 按日记检索才这样翻页
        data = browser.page_source
        soup = BeautifulSoup(data, 'html.parser')
        pages = soup.select('.right.pagination.hidden a')
        position = len(pages)
        # 没用，结束不了
        # if browser.find_element_by_xpath('// *[ @ id = "pagination"] / a[8]') == None:
        #     break

        Total_pagenum = int(soup.select('.right.pagination.hidden a')[-2].text)
        if page >= Total_pagenum:
            break

        #browser.find_element_by_xpath(f'// *[ @ id = "pagination"] / a[{position}]').click()
        browser.find_element(By.XPATH, f'// *[ @ id = "pagination"] / a[{position}]').click()
        time.sleep(3)

        data = browser.page_source
        soup = BeautifulSoup(data, 'html.parser')
        Items = soup.select('.m_feed_item')
        for item in Items:
            try:
                l = []
                item = str(item)
                soup = BeautifulSoup(item, 'html.parser')
                # 多了个m_qa就查不到，奇怪，但是源码里本来就有m_qa呀，不知道select怎么匹配的
                # look = soup.select('.m_feed_detail m_qa')
                Info = soup.select('.m_feed_detail .m_feed_face a')
                Text = soup.select('.m_feed_detail .m_feed_cnt .m_feed_txt')
                Time = soup.select('.m_feed_detail .m_feed_from span')

                # 如果有答复
                if len(Info) == 2:
                    Ques_per_name = Info[0]['title']
                    Ques_per_id = Info[0]['uid']
                    Ques_per_url = 'http://sns.sseinfo.com/' + Info[0]['href']
                    location_url = requests.get(Ques_per_url, timeout=5).text
                    Location = re.findall('(?<=所在地：).*?(?=[<])', location_url)[0]
                    item_2 = Ques_per_name
                    item_3 = Ques_per_id
                    item_4 = Location
				
                    Ques_per_text = Text[0].text
                    Ques_per_text = Ques_per_text.strip()
                    Ques_per_text = re.sub('<.*?>', '', Ques_per_text)
                    Ques_per_text = re.sub(',', '', Ques_per_text)
                    Ques_per_text = re.sub(' ', '', Ques_per_text)
                    Ques_per_text = re.sub('\n', '', Ques_per_text)
                    
                    item_0 = re.findall('.*?(?=[(])', Ques_per_text)[0][1:]
                    item_1 = re.findall('(?<=[(]).*?(?=[)])', Ques_per_text)[0]

                    Ques_per_text = re.sub('.*?[)]', '', Ques_per_text)
                    item_5 = Ques_per_text

                    item_6 = f'{year}年' + Time[0].text

                    Ans_per_name = Info[1]['title']
                    Ans_per_id = Info[1]['uid']
                    Ans_per_context = Text[1].text
                    Ans_per_context = Ans_per_context.strip()
                    Ans_per_context = re.sub('<.*?>', '', Ans_per_context)
                    Ans_per_context = re.sub(',', '', Ans_per_context)
                    Ans_per_context = re.sub(' ', '', Ans_per_context)
                    Ans_per_context = re.sub('\n', '', Ans_per_context)
                    item_7 = Ans_per_context

                    print(str(int(item_1))+f' {Ques_per_name} {Ques_per_id} {Ques_per_text}')
                    print(f'{Ans_per_name} {Ans_per_id} {Ans_per_context}')

                    l.append(item_0)
                    l.append(item_1)
                    l.append(item_2)
                    l.append(item_3)
                    l.append(item_4)
                    l.append(item_5)
                    l.append(item_6)
                    l.append(item_7)

                    tmpstr = ','.join(l) + '\n'
                    to_csv.write(tmpstr)

                # 如果没回复
                if len(Info) == 1:
                    Ques_per_name = Info[0]['title']
                    Ques_per_id = Info[0]['uid']
                    Ques_per_url = 'http://sns.sseinfo.com/' + Info[0]['href']
                    location_url = requests.get(Ques_per_url, timeout=5).text
                    Location = re.findall('(?<=所在地：).*?(?=[<])', location_url)[0]
                    item_2 = Ques_per_name
                    item_3 = Ques_per_id
                    item_4 = Location

                    Ques_per_text = Text[0].text
                    Ques_per_text = Ques_per_text.strip()
                    Ques_per_text = re.sub('<.*?>', '', Ques_per_text)
                    Ques_per_text = re.sub(',', '', Ques_per_text)
                    Ques_per_text = re.sub(' ', '', Ques_per_text)
                    Ques_per_text = re.sub('\n', '', Ques_per_text)
                    item_0 = re.findall('.*?(?=[(])', Ques_per_text)[0][1:]
                    item_1 = re.findall('(?<=[(]).*?(?=[)])', Ques_per_text)[0]

                    Ques_per_text = re.sub('.*?[)]', '', Ques_per_text)
                    item_5 = Ques_per_text

                    # 后面再去测试
                    item_6 = f'{year}年'+Time[0].text
                    item_7 = '无'

                    l.append(item_0)
                    l.append(item_1)
                    l.append(item_2)
                    l.append(item_3)
                    l.append(item_4)
                    l.append(item_5)
                    l.append(item_6)
                    l.append(item_7)

                    tmpstr = ','.join(l) + '\n'
                    to_csv.write(tmpstr)

            except:
                print('文本异常，忽略这条')
                continue

    to_csv.close()

#if __name__ == '__main__':
#    Due_Time(2021, 3)



In [21]:
for m in range(10,13): # for other
    Due_Time(2013, m)

1
600059 JSNH 10484 既然是公司投入巨资拍女儿红，为什么第一年的版权收益又不归公司呢？
古越龙山 59 您好，公司拍摄《女儿红》的宗旨为努力扩大广告宣传之效能，播出发行和努力扩大发行面为首要目的，播出越多品牌效应越强，首先要确保播出发行。感谢您的留言！
600059 JSNH 10484 公司的资产负债率不到30%为什么融资不选择发债，而要配股呢？
古越龙山 59 您好，公司综合考虑各种因素决定以配股方式募集资金，投资兴建黄酒生产技术升级项目及第二期“百城千店”专卖店终端网络项目。感谢您的留言！
600155 王者风范 10215 工作效率太慢，相关土地拍卖结果已经公示，公司公告迟迟未见，违背上市公司信息披露原则，
*ST宝硕 155 谢谢您对我公司的关注，您的鞭策和鼓励，都将是我们自我改进和提升动力。
600313 amanofsky 5420 请问中农是否会放弃对锦绣华农的收购？
中农资源 313 中垦总公司在2012年5月16日作出承诺，尚在承诺期内，公司已督促中垦总公司按时履行承诺，目前中垦总公司正在进行此项工作。感谢投资者关注。
600313 amanofsky 5420 请问上市公司有没有义务回答股东关于公司土地的问题。做为中农的股东，为何不能够知道公司所拥有的相关资产，土地以及租金等问题？机构作为非公司股东，到上市公司调研是否可以获得更详细的土地信息？
中农资源 313 公司会按照监管机构规定履行信息披露的义务，公司对机构的信息披露亦会遵循监管机构要求，感谢投资者关注。
600313 amanofsky 5420 大股东承诺11月16日前转让锦绣华农股权给上市公司。请问上述承诺日期指的是股权注入完成日期还是股权注入启动日期？若大股东违背承诺，受到监管部门关注，是否对明年的再融资计划有影响？
中农资源 313 中垦总公司在2012年5月16日作出承诺，尚在承诺期内，公司已督促中垦总公司按时履行承诺，目前中垦总公司正在进行此项工作。公司会根据实际发展需要利用上市公司平台提供融资支持。感谢投资者关注。
600313 amanofsky 5420 三季报中说明今年业绩与去年同期相比重大变动警示：不适用，请问，此重大变动指的是什么？30%还是50%
中农资源 313 《公开发行证券的公司信息披露编报规则第13号----季度报告内容与格式特别规定

600883 投资者10440 10440 东财股吧和上证互动真热闹，本主也来凑一份，弱弱地问一句，股东的心声董事们，高层领导们听到了吗？行动了吗？
博闻科技 883 衷心感谢您对公司的关注！
7
600056 shuguilin 9675 公司短期贷款将近20亿，财务费用激增，一年利息就是个惊人的数字，这个问题怎么解决？
中国医药 56 公司将通过资本市场、业务整合、内部调配等方式保持健康财务水平。
600056 shuguilin 9675 公司重组时曾发布预测盈利报告，称2013年有望实现营收168亿，净利5.2亿，可是三季报只实现营收109亿，净利3.2亿，请问如可保证2013年原定目标的实现？
中国医药 56 重组盈利预测是公司2013年既定目标，公司正全力完成。
600056 shuguilin 9675 公司三季报预告2013年业绩预增25%至35%，请问2012年合并报表后的业绩是多少？三季报里没有反映。
中国医药 56 2012年合并归母净利润约4.16亿元。
600056 shuguilin 9675 基金季报显示原持仓绝大多数基金选择逃离中国医药，最近有机构到公司调研吗？
中国医药 56 公司与机构投资者的日常接触均正常开展中。
600715 天下攘攘 10219 22日凌云B股发布的公告中提到从德祐能源科技有限公司原股东手中买到了95%的股权，这不是松辽汽车的下属公司吗？并且是本次增发方案中准备拟11.4亿元投建100MW的项目。既然本项目已经出售，为什么没有公告？出售后已经造成原增发方案大幅变动，为什么没有公告？
松辽汽车 715 公司近期已经披露相关公告，敬请关注。
600056 shuguilin 9675 三季报显示公司三季度管理费用达8000多万，同比增幅达40%多，系重组费用，请问具体是哪些费用？重组需要些什么费用？
中国医药 56 重组所发生的中介费用包括财务顾问费、法律顾问费、会计师事务所费用以及资产评估费用等。
600056 shuguilin 9675 根据三季报推算，除海南三洋外，武汉鑫益、新疆天方等收购的子公司几乎没有创造效益，请问公司将采取什么措施帮助这些公司开拓业务？
中国医药 56 如三季报所言，公司正加大整合梳理力度。另外，并购子公司并非没有创造效益。
600366 我是投资者 5726 接着8月股民代表的问题

600240 沟通才有信任 8152 请问，楼面价11000元/平方米，房子一般买多少钱？谢谢
华业地产 240 投资者您好，房价受很多因素影响，如地价、建安成本、工程管理费、市场行情等，因此公司无法以楼面价预估房价。
600240 沟通才有信任 8152 习近平重申3600万套保障房要全力完成。公司位于海南三亚旧城改造或别的项目，是否从中获益？谢谢
华业地产 240 投资者您好，公司将密切关注后续进展，结合实际情况，研究对公司发展带来的机遇。
600754 danny 7074 锦江之星以单店特许经营的方式进军韩国，目前进展如何？位于韩国首尔市中区的锦江之星今年底能不能开业？多谢！
锦江股份 754 该项目尚在筹备中，感谢您对公司的关注。
600754 12345chen 10021 你们参股申银万国多少股.现在在账面上它的成本价是多少.申银万国最近有否动作能否上市.你们会否增持.谢谢
锦江股份 754 公司持有申银万国证券股份有限公司11088566股，最初投资金额1000万元。关于申银万国上市，公司也是通过媒体报道获悉相关信息。公司会密切关注后续事宜，感谢您对公司的关注。
600754 12345chen 10021 你们与微信合作想问下与它的合作对收益能有多少帮助. 我感觉只是个概念实质利好也不是多大吧.谢谢
锦江股份 754 锦江之星总部微信服务号于2013年9月20日上线即预订功能开通，目前在整个中央渠道预订量中的占比还非常小。感谢您对公司的关注。
600267 xiaofengzi123 7001 你好，请问公司新申报的AD-35片是个什么品种？适应症等相关信息请多提供一点，谢谢。
海正药业 267 您好！AD-35片现仅完成临床前研究，处于非常早期的阶段。谢谢！
600267 文章77 10360 请问三季度的各项费用及税金为什么环比增长很多？而销售收入环比二下降6%。
海正药业 267 您好！税金及附加是=（增值税+营业税+消费税）*12%来计算，增值税=销项税款-进项税款，如当季度进项税款减少，会导致税金及附加增加。谢谢！
600521 小随便 5884 就公司今年的业绩都不达预期，如果让股东相信所谓的十二五规划？兼并喊了大半年，啥都没有？制剂出来喊啦快一年，啥都没有？原料药还是负增长？请问公司今年在忙个啥东西？为股东创造了啥价值？
华海药业 5

600598 安民1 5863 本届董事会已超期，公司何时进行董事会改选？
北大荒 598 公司第五届董事会第二十九次会议中包含换届改选的议题。
600110 superv777 9128 公司经营范围主业是锂电池用的铜箔和新型橡胶，是否收购稀土生产稀土异戊橡胶，请正面回答
中科英华 110 公司主营业务是铜箔、电线电缆及附件、电池材料，公司暂未考虑是否要生产稀土异戊橡胶。收购稀土矿资产完成后，公司将结合新材料产业链布局充分利用稀土资源。
600598 龙山风影 9823 请问，公司现有耕地多少亩？水田、旱田、可开垦的土地分别有多少亩？ 据说，公司的耕地都是属于大股东的，是吗？
北大荒 598 关于公司现有耕地相关信息，公司已公告过，国家禁止开垦荒地；公司耕地是通过承包形式从大股东获得的耕地。
600110 公子战博客 7734 为什么要搞稀土增储？因为之前相对于西藏发展属于超高溢价！现在弄一个稀土储量增长，无非是拿这个当做借口，其实如果是真对上市公司有利，应该是先压低价格购买，然后再自己搞增储！大股东就是想借此给人以更有正当性，实际上增储后就要多付出资金给原有大股东，虽然有价格上限，但逼近上限无疑不好。
中科英华 110 您好！您对增储一事以及在增储的实施者上有误解，增储后收购是双方谈判的结果，目前厚地稀土的勘探工作及地质报告编审工作仍由原公司经营团队负责开展，待以上工作完成后的评估工作才是由公司牵头进行，评估工作后才会根据评估报告产生收购价格，从公司的角度自然希望以低价格收购，但这要以评估报告数据为依据并由双方协商产生。
600725 WalMart1918 5402 您好！请问一下，近期3个月内贵公司董事会是否会考虑进行重组，从而提升公司市场竞争力，还望咨询，谢谢！
云维股份 725 答：受宏观经济影响，近年来公司的经营业绩不理想，公司采取了各种措施努力改善经营业绩，进行了资产出售、物流资产重组等工作，对组织机构和管控模式进行了改革，取得了明显效果，公司会根据自身发展实际和环境变化，积极调研，深入分析，科学决策，采取各种措施，提升公司市场竞争力，促进公司健康发展。
600725 金犇投资集团 10311 您好，随着国研中心383改革方案公布，金融改革继续深入，国内已放宽民营银行的设立，目前煤企融资渠道单一，金犇投资集团正在积极运作筹建煤炭银行，请问贵公司是否有

600390 巴哈马 10135 谢谢你对投资者的耐心解答读到贵公司回答时有几点不解请问：金天材料公司现有几条前躯体生产线分别设计产能多少?NCA与NCM2013年上半年产销量是多少?营收多少?金驰现在进度怎么样?公司是否会收购大股东持有的金驰余下股权?
金瑞科技 390 公司现有NCA与NCM前躯体生产线各一条NCA设计年产能1000吨，NCM经过上半年的改扩后年产能达到1500吨。2013年上半年NCA产品根据订单生产，月产量不超过30吨；NCM上半年的销售量350吨左右。 金驰公司10000吨锂电池正极材料项目目前已完成工程项目监理招标工作和工程设计前期工作，土地的征收及报建、报批工作正在进行过程中。 电源材料作为上市公司的主业之一，大股东把集团内所有电源材料资产纳入上市公司是集团战略选择，但是目前没有确定金驰公司剩余股权注入上市公司的方式及时间。
600239 z76075 5343 请问城投公司在云南桥头堡建设过程中作用和昆明保税区的建设对公司发展的影响，环湖东路续约工作进度，大致的方案是什么？可双方的分歧在哪里，何时可以定案（公司既然认为不是重大事项简单说一下应该可以的吧）。
云南城投 239 您好，云南桥头堡建设对云南省经济发展的影响是多方面的，可能会给公司带来更多的机遇但具体效果现在还难以评估。环湖东路项目公司仍还在与政府进行积极的磋商，有结果我们会第一时间披露，感谢您的关注。
600110 跟随者 9994 希望公司加强电缆、铜箔、锂电池、稀土等销售及资金回笼，提高效率。
中科英华 110 感谢您对中科英华的支持。公司始终结合市场情况强化营销策略，推进销售实现和回款，稀土产品因尚未完成收购事项目前无法确认销售收入。
600817 喜羊羊 10290 请问根据法律规定，在公司资产处置及债务清偿等工作完成前，公司是否可以与第三方进行资产重组？强烈建议公司恢复和赣州稀土的谈判 给亏损累累的小散一个交代. 请回答.谢谢
ST宏盛 817 您好：关于公司破产重整工作与重组事项，公司与山西天然气公司开展重组合作时，公司正处于重整工作执行期，虽然最终重组事项终止，但可以看出重整工作与重组工作是可以同时进行的。 至于与赣州稀土的合作终止公司已经公告，请广大投资者注意投资风险！
26
600110 wxc 10064 中科英华为何叫“骗子公司”：1、公司与清华大学联

...

600056 shuguilin 9675 公司股份长期在20元上下，甚至低于增发价，说明二级市场投资者也不看好公司的后市。如果说是短期不看好也就算了，为什么三四年以来公司的股价一直都在20元这个平台附近不动，公司自己认为发展的最大软肋是什么？
中国医药 56 尊敬的投资者：今年公司完成了重大资产重组，吸收了大量的医药工业和新的医药商业资产，进一步完善了以国际贸易为引领，以医药工业为支撑，以医药商业为纽带的科工贸一体化企业架构。未来的首要任务就是将内部资源整合好，战略规划好，外生增长扎实推进。
600088 ctv-少林方丈 10494 请问梁董事长，12月9日开盘后，600088股价如果连续几个跌停，我是23.6元买的，将面临巨额亏损，我的损失梁董事长负责赔偿吗？10月9日买的，看到了重组公告利好才买的，9月28日的公告还是说正在推进中，一切正常，10月18日突然说终止重组，我被欺骗了，巨额损失该谁负责？！
中视传媒 88 投资者您好！此次资产重组过程，公司严格按照《上市公司重大资产重组管理办法》、《上市公司重大资产重组信息披露工作备忘录》等相关规定，充分关注重组事项的进展，及时履行信息披露义务，并且在重组预案和历次进展公告中均客观地提示风险，提请广大投资者谨慎投资，符合信息披露的要求。
600056 shuguilin 9675 请问在与机构投资者沟通的过程中，机构投资者为什么会不看好公司的长远发展，它们最大的担心和忧虑是什么呢？
中国医药 56 尊敬的投资者：从前期的沟通来看，主要问题还是由于和市场的沟通交流仍不够充分，部分机构对公司业务的了解不多，甚至认识有误差，存在对公司的盈利模式看不清等问题。经过逐家的深入沟通，这些问题已逐渐得到解决。
600056 shuguilin 9675 除委内、古巴外，公司在其它亚非拉第三世界国家的发展如何，未来有无可能收获类似委内项目61亿的大单？
中国医药 56 尊敬的投资者：公司始终对发展中国家市场保持关注，并通过集团、公司的代表处以及公司的业务部门进行广泛的项目搜索的业务接触。另外，公司也一直在加强挖掘大项目的工作。
600216 zdp 5783 请问你们证代在回答问题前，有没有读过自己公司的年报？？？？？？ 浙药2012年年报第87页倒数第2行 人力资源2000000.00 ... 数额是不大。就是不明白这是个什么

600804 Teyoung 7641 电信通网页上看到大麦盒子到货了？送盒子的活动好像11月就有了，对公司大宽带的销售是否有推动？盒子确实如市场猜测是创维代工的，创维的logo貌似太大了，看着就以为是创维的盒子，而不是DomyBox。
鹏博士 804 捆绑大麦高清盒子肯定对公司大带宽产品的销售有很大促进。盒子logo的问题公司考虑下，谢谢您的提醒。
273
600110 laotaitai 10000 公司将召开第八次临时股东大会，股民为对会议提出建议需了解一基本情况：德昌大陆槽3号矿体是属于公司的探矿矿体吗？请公司及时正面告知。
中科英华 110 是的。
600110 黄金甲888 9960 1。公司非公开发行股票收购厚地稀土进展如何？证监会批准没有？批准有难度吗？2。如果稀土储量大幅增加，公司收购厚地稀土会有违约风险吗？违约概率有多大？3.公司有没有派人员到厚地稀土公司任职？厚地稀土的管理现在是中科英华在决策吗？4.公司收购厚地稀土现在存在哪些问题和困难？
中科英华 110 您好，对您一口气问的这么多问题逐一回复如下：1、目前，公司本次非公开发行股票购买资产事项相关审计、评估工作仍持续进行，尚未上报证监会。2、如果稀土储量大幅增加，16.88亿也是公司收购价格的上限。3、目前，厚地稀土的日常管理由其原经营层负责。4、公司收购厚地稀土项目中，有些事宜会涉及政府部门，这类事宜相关文件的审批和办理可能要花一定的时间。
600110 戒贪戒嗔 10024 公司在11月9日公告说，大陆槽稀土矿的勘探报告编写进入尾声，目前是否已经完成，是否已经上报四川国土资源厅？
中科英华 110 目前，地质队正在准备内部审核工作，将在地质报告定稿并经专家评审后履行上报四川省国土资源厅程序，目前尚未上报。公司将在地质报告上报四川省国土资源厅程序有实质进展后进行及时公告。
600110 laotaitai 10000 马上召开第八次临时股东大会，希望股东有“宝贵的意见或建议”，在提出意见和建议之前，股东需了解一些基本情况，为什么这么多的股东提问不及时答复，是对股东有诚意的表现吗？？
中科英华 110 您好，您的问题我们已回复，谢谢。
600804 Teyoung 7641 请问：截止11月末公司的在网用户数是多少啊？
鹏博士 804 公司目前每月的用户增长在15万左右。11月末的用户数因未

600538 天天赢 9336 如果今年年底只收到第一期2500万，那么年报可以确认收益嘛？年报可以扭亏嘛？
*ST国发 538 11月23日，公司已披露了2013年度业绩预盈公告，请查看上海证券交易所网站公司的公告。政府收回公司土地使用权的补偿款，按会计准则的相关规定，可以确认为2013年的收益。
279
600420 shuguilin 9675 公司从大股东上海医工院手里买了那么多合作科研项目，但几乎都没有转化为现实的热销产品，倒是卖给海正等公司之后人家做得风生水起，请问公司有无反思原因何在？
现代制药 420 尊敬的投资者，感谢您对公司的关注。公司与控股股东上海医药工业研究院转让技术项目均按照计划稳步推进，部分产品或处于临床研究阶段，或处于生产报批阶段，还有部分产品已上市销售，之前上市品种请参看公司历年年度报告。
600420 shuguilin 9675 上海市发改委日前公布上海市医疗机构第二轮基本药物限价通知，涉及现代制药2款药，请问是哪两个品种，对公司业绩造成何种负面影响？
现代制药 420 尊敬的投资者，感谢您对公司的关注。本轮基本药物限价涉及142个产品中包括现代制药的格列齐特片（80mgX60）及非那雄胺片（5mgX20）。以上两个品种占公司总体销售收入比例不高，将不会对公司生产经营产生重大影响。
600420 shuguilin 9675 公司股价低迷不振，请问高管准不准备增持以示信心？
现代制药 420 尊敬的投资者，感谢您对公司的关注。对公司的股价取决于不同投资者的判断。
600420 shuguilin 9675 现代制药新领导班子上任之后，对公司有何全新的规划和设想？准备通过何种渠道如何实现飞跃式发展？
现代制药 420 尊敬的投资者，感谢您对公司的关注。公司总经理的变动对公司战略发展规划无影响，公司战略规划为：以非头孢类、非青霉素类药品为主攻方向，研究和生产有特色的原料药和制剂，沿既有的产品路线，大力拓展其它市场，致力于成为国内领先的研发驱动型制药企业。
600420 路弯弯 12810 控股子公司的尿促卵泡素获得中国专利金奖，据说是原料药？跟处方药、OTC药物有什么不同？国内独家生产，其市场规模有多大？既然能获得专利金奖，应该能够提升上市公司的盈利能力吧？谢谢
现代制药 420 尊敬的投资者，感谢您对公司的关注。原料药是用于生产制剂

600567 彼岸的新房客 10682 希望简要的了解一下，贵司在专业角度、行业参与者的角度，对当前造纸行业现状的分析以及未来前景的判断。谢谢。
山鹰纸业 567 近年来，造纸及纸制品行业总体情况为，产销增速双双放缓，企业库存压力有所好转，行业供求状况趋于稳定，利润增速止跌回稳。未来在原材料成本下降和淘汰落后产能的促进下，造纸行业利润空间将逐步回升。
600567 问问看看 12399 80万吨新项目建设期4年对于公司来说时间太长，时间就是金钱（银行利息以及），就是商机。公司需要大力扩大公司及品牌的知名度和影响力（经常能在媒体看到九龙和理文的新闻和报道，而公司的则很少），公司现在已经不再是区域性的了。
山鹰纸业 567 为了确保新项目能够尽快投产见效，公司一直在积极努力推进80万吨造纸项目的建设工作。非常感谢您的关注与建议！
601098 许卓苗 10751 您好！请问截止11月底股东人数是多少？谢谢。
中南传媒 1098 您好！公司按相关规定在定期报告中披露报告期末股东户数，请届时查阅，谢谢！
600900 小陈爱长江 10269 中国部署全国碳交易市场，希望公司关注并加快国内以及国际的水电CDM项目的注册，以实现水电作为清洁能源的环保价值，增加公司收益。
长江电力 900 尊敬的投资者，感谢您对长江电力的关注！公司目前没有参与CDM交易，但在国务院下发的《关于加快培育和发展战略性新兴产业的决定》中首次提到，要建立和完善主要污染物和碳排放交易制度，有关部门也表示，未来将选择部门、行业和省份开展碳交易试点。公司会积极关注。您的建议我们会及时转呈公司董事会和管理层。公司会根据相关监管规定及时、准确地向全体股东披露有关信息。
600056 xizhuxi 10127 7、今天IPO版本发行意见，1月份开闸。对于公司来说，应尽快召开中介机构协调会，动用各方资源，甚至大股东的资源，全力推进增发，力争在1月前完成，不然1月后资金宁愿参与新股网下配售，对增发兴趣降至冰点（历史经验）。对公司来说现在既是机遇也是挑战。
中国医药 56 尊敬的投资者：您的考量公司也有体会。感谢您的建议！
600056 xizhuxi 10127 6、如有可能，建议公司在近期组织医药研究员反向路演，事先将路演资料发出，现场讲解不要太多时间，提问环节可稍多一些时间，在重点问题上和大家多探讨。不要企图说服

600100 快乐的慧剑心魔 10002 你好，请问公司截止到11月31日股东人数为多少？另外北京司马台地区采用同方人环的热泵采暖技术受到良好的评价，是不是意味着这块业务将来几年会有一个爆发式的增长？
同方股份 100 您好！截至11月底，公司股东总数大概在15.5万户左右。热泵技术一直是公司在节能环保领域的一项核心技术，以热泵、余热回用等技术为核心的节能业务也一直是公司近年来重点培育的产业之一，目前公司正在积极开展有关市场推广和商业模式创新工作，并已经推动了一批合同能源管理项目的实施。
600858 lw667 12660 为什么银座商城可以开银座网，而银座股份没有？银座商城的银座网销售的母婴产品是否与银座股份的商品冲突？是不是同业竞争？请解释。
银座股份 858 公司第一大股东山东省商业集团有限公司将本公司作为未来整合其所控制的零售业务资源的唯一主体。银座网是大股东旗下零售业为积极开展电子商务、探索将线下资源与线上业务进行融合而采取的有益尝试，尚在培育与探索。银座网目前经营品类较少，其品种构成及顾客群体等与银座实体店存在差别。谢谢关注。
600563 qinai 8974 你好，请问公司是否有和军工企业深入合作，直接提供电容器呢？
法拉电子 563 公司没有给军用产品提供薄膜电容器
600858 lw667 12660 为什么银座商城所发的购物卡可以在银座股份所属门店使用，而银座股份不能发购物卡在银座商城使用？请解释。
银座股份 858 您提及的购物卡是公司第一大股东山东省商业集团有限公司旗下零售，在重组本公司前身ST渤海并注入部分优质零售资产之前即已发行使用的，对公司门店是一种像银行卡一样的结算方式。公司未发行购物卡。谢谢关注。
290
600891 bluefish958 12798 请问公司的两件诉讼进展如何?最晚什么时候能出结果?还有三月份的非公开发行为何没有下文了?
秋林集团 891 bluefish958，您好！目前，两起诉讼案正在等待判决。非公开发行请您继续关注公司的公告。
600891 老简股友 6166 不知道证监会和上交所对屡屡不兑现承诺的秋林集团是否有制裁措施和处理办法呢？
秋林集团 891 老简股友，您好！承诺事项所涉的大新街两处房产的实质控制权和出租收益一直归上市公司所有。上市公司派专人负责房产出租、物业管理等工作。因历史原因形成的两处